In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
torch.device("cuda")

device(type='cuda')

In [36]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from collections import deque, namedtuple
import copy

In [49]:
class Brain(nn.Module):
    def __init__(self, input_shape, output_size):
        super(Brain, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)

        # Calculate the size of the output from the last conv layer
        conv_out_size = self._get_conv_output(input_shape)

        # Fully connected layers
        self.fc1 = nn.Linear(conv_out_size, 512)
        self.fc2 = nn.Linear(512, output_size)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        output = self.conv3(self.conv2(self.conv1(input)))
        return int(np.prod(output.size()))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x))

In [50]:
import gymnasium as gym
env = gym.make("CarRacing-v2")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_space = env.action_space
print("State shape: ", state_shape)
print("state size: ", state_size)
print("number of actions: ", action_space)

State shape:  (96, 96, 3)
state size:  96
number of actions:  Box([-1.  0.  0.], 1.0, (3,), float32)


In [51]:
learning_rate = 1e-3  # Slightly increased for faster learning
minibatch_size = 64   # Reduced to a more standard size
discount_factor = 0.99
replay_buffer_size = int(1e5)
tau = 1e-3  # Soft update parameter (changed from 'tao' for conventional naming)

In [52]:
class ReplayMemory:
    def __init__(self, capacity):
        self.device = torch.device("cuda:0")
        self.capacity = capacity
        self.memory = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        experiences = random.sample(self.memory, k=batch_size)
        
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)

In [53]:
class Agent():
    def __init__(self, state_shape, action_space):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_shape = state_shape
        self.action_size = action_space.shape[0]
        
        # Using actor-critic approach for continuous action space
        self.actor = Brain(state_shape, self.action_size).to(self.device)
        self.critic = Brain(state_shape, 1).to(self.device)
        self.target_actor = Brain(state_shape, self.action_size).to(self.device)
        self.target_critic = Brain(state_shape, 1).to(self.device)
        
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=learning_rate)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=learning_rate)
        
        self.memory = ReplayMemory(replay_buffer_size)
        self.t_step = 0
        
        self.noise = OUNoise(self.action_size)

    def act(self, state, add_noise=True):
        state = torch.from_numpy(state).float().unsqueeze(0).permute(0, 3, 1, 2).to(self.device)
        self.actor.eval()
        with torch.no_grad():
            action = self.actor(state).cpu().data.numpy()
        self.actor.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1, 1)

    def learn(self, experiences, discount_factor):
        states, actions, rewards, next_states, dones = experiences

        # Update critic
        next_actions = self.target_actor(next_states)
        Q_targets_next = self.target_critic(torch.cat([next_states, next_actions], dim=1))
        Q_targets = rewards + (discount_factor * Q_targets_next * (1 - dones))
        Q_expected = self.critic(torch.cat([states, actions], dim=1))
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Update actor
        actions_pred = self.actor(states)
        actor_loss = -self.critic(torch.cat([states, actions_pred], dim=1)).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks
        self.soft_update(self.actor, self.target_actor, tao)
        self.soft_update(self.critic, self.target_critic, tao)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

class OUNoise:
    def __init__(self, size, mu=0., theta=0.15, sigma=0.2):
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.reset()

    def reset(self):
        self.state = copy.copy(self.mu)

    def sample(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

In [47]:
agent = Agent(state_shape, action_space)

In [55]:
number_episodes = 2000
max_timesteps_per_episode = 1000
scores_window = deque(maxlen=100)

for episode in range(1, number_episodes + 1):
    state, _ = env.reset()
    print(f"Initial state shape: {state.shape}")
    agent.noise.reset()
    score = 0
    
    for t in range(max_timesteps_per_episode):
        action = agent.act(state)
        print(f"Action shape: {action.shape}, Action: {action}")
        next_state, reward, done, _, _ = env.step(action)  # Remove [0] indexing
        print(f"Next state shape: {next_state.shape}, Reward: {reward}, Done: {done}")
        agent.step(state, action, reward, next_state, done)  # Remove [0] indexing
        state = next_state
        score += reward
        if done:
            break
    
    scores_window.append(score)
    
    print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_window):.2f}')
    if episode % 100 == 0:
        print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_window):.2f}')
    
    if np.mean(scores_window) >= 200.0:
        print(f'\nEnvironment solved in {episode-100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
        torch.save(agent.actor.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic.state_dict(), 'checkpoint_critic.pth')
        break

Initial state shape: (96, 96, 3)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (9216x3 and 27648x400)